In [67]:
import pandas as pd
import datetime
import numpy as np
import pickle
import os
import pymongo
import requests
import json
from datetime import date
from datetime import timedelta 
import unicodedata

## LOADING DATA

In [68]:
concats = pd.DataFrame()
for i in range(14,20):
    concats = pd.concat([concats , pd.read_csv("../data/"+str(i)+"01.csv", encoding="ISO-8859–1")], sort = False,ignore_index=True)
    
for i in range(14,20):
    concats = pd.concat([concats , pd.read_csv("../data/"+str(i)+"02.csv", encoding="ISO-8859–1")], sort = False,ignore_index=True)

In [69]:

for i in range(14,20):
    concats = pd.concat([concats , pd.read_csv("../data/"+str(i)+"03.csv", encoding="ISO-8859–1")], sort = False,ignore_index=True)
    
for i in range(14,20):
    concats = pd.concat([concats , pd.read_csv("../data/"+str(i)+"04.csv", encoding="ISO-8859–1")], sort = False,ignore_index=True)

In [70]:

for i in range(14,20):
    concats = pd.concat([concats , pd.read_csv("../data/"+str(i)+"05.csv", encoding="ISO-8859–1")], sort = False,ignore_index=True)
    
for i in range(14,20):
    concats = pd.concat([concats , pd.read_csv("../data/"+str(i)+"06.csv", encoding="ISO-8859–1")], sort = False,ignore_index=True)

In [71]:

for i in range(14,20):
    concats = pd.concat([concats , pd.read_csv("../data/"+str(i)+"07.csv", encoding="ISO-8859–1")], sort = False,ignore_index=True)
    
for i in range(14,20):
    concats = pd.concat([concats , pd.read_csv("../data/"+str(i)+"08.csv", encoding="ISO-8859–1")], sort = False,ignore_index=True)

In [72]:

for i in range(14,20):
    concats = pd.concat([concats , pd.read_csv("../data/"+str(i)+"09.csv", encoding="ISO-8859–1")], sort = False,ignore_index=True)
for i in range(14,20):
    concats = pd.concat([concats , pd.read_csv("../data/"+str(i)+"10.csv", encoding="ISO-8859–1")], sort = False,ignore_index=True)

In [73]:
concats

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,HFKC,AFKC,Referee
0,F1,08/08/14,Reims,Paris SG,2.0,2.0,D,2.0,1.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,F1,09/08/14,Bastia,Marseille,3.0,3.0,D,1.0,2.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,F1,09/08/14,Evian Thonon Gaillard,Caen,0.0,3.0,A,0.0,3.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,F1,09/08/14,Guingamp,St Etienne,0.0,2.0,A,0.0,1.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,F1,09/08/14,Lille,Metz,0.0,0.0,D,0.0,0.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22195,I2,01/02/2020,Empoli,Crotone,3.0,1.0,H,2.0,1.0,H,...,1.83,2.05,1.85,2.07,1.86,2.01,1.82,NaN,NaN,NaN
22196,I2,02/02/2020,Chievo,Venezia,0.0,1.0,A,0.0,1.0,A,...,2.00,1.89,2.01,1.89,2.05,1.84,1.98,NaN,NaN,NaN
22197,I2,02/02/2020,Juve Stabia,Perugia,1.0,2.0,A,1.0,1.0,D,...,1.93,1.93,1.96,1.97,1.97,1.91,1.91,NaN,NaN,NaN
22198,I2,02/02/2020,Benevento,Salernitana,1.0,1.0,D,0.0,0.0,D,...,2.02,1.86,2.04,1.89,2.10,1.81,2.02,NaN,NaN,NaN


In [74]:
concats.columns[:22]

Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY',
       'AY', 'HR', 'AR'],
      dtype='object')

## Data engineering

In [75]:
def stringToDate(string):
    string = string.split("/")
    if(len(string[2]) == 4):
        string[2] = string[2][2:]
    date = string[0]+"/"+ string[1]+"/20"+string[2]
    date = datetime.datetime.strptime(date, '%d/%m/%Y')
    return date

In [76]:
def hash_match(date, away, home, ver):
    date = date.split("/")
    return date[0]+date[1]+date[2]+away+home+str(ver) 

In [77]:
def getLastMatchId(id_match, nb_earlier_match):
    dates = concats['Date'].apply(stringToDate)
    #homepart
    ids_HH = dates[(dates < dates[id_match]) & (concats["HomeTeam"] == concats["HomeTeam"][id_match])].index
    ids_HA = dates[(dates < dates[id_match]) & (concats["AwayTeam"] == concats["HomeTeam"][id_match])].index
    ids_home = np.array(ids_HH,ids_HA)
    if(ids_home.shape[0] < nb_earlier_match):
        return np.nan
    else:
        ids_home = ids_home[ids_home.argsort()[-nb_earlier_match:][::-1]]
    #awaypart
    
    ids_AH = dates[(dates < dates[id_match]) & (concats["HomeTeam"] == concats["AwayTeam"][id_match])].index
    ids_AA = dates[(dates < dates[id_match]) & (concats["AwayTeam"] == concats["AwayTeam"][id_match])].index
    ids_away = np.array(ids_AH,ids_AA)
    if(ids_away.shape[0] < nb_earlier_match):
        return np.nan
    else:
        ids_away = ids_away[ids_away.argsort()[-nb_earlier_match:][::-1]]
    return np.array([ids_away, ids_home])

In [78]:
concats = concats[pd.notnull(concats['FTR'])]
concats = concats.drop(columns = concats.columns[23:])
concats['ID_FTR'] = concats.FTR.astype('category').cat.codes

In [79]:
concats = concats.drop(columns = concats.columns[9])

In [80]:
hashmap = {}

In [81]:
try:
    with open('../data/hm.pickle', 'rb') as handle:
        hashmap = pickle.load(handle)
except FileNotFoundError as err:
    print("file not yet created")

In [82]:
ids = []
for i in range(concats.shape[0]):
    vals = concats.iloc[i,[1,2,3]].values
    hashed = hash_match(vals[0],vals[1], vals[2], 1)
    if hashed in hashmap:
        ids.append(hashmap[hashed])
    else:
        id_match = getLastMatchId(concats.index[i],3)
        ids.append(id_match)
        hashmap[hashed] = id_match


In [83]:
with open('../data/hm.pickle', 'wb') as handle:
    pickle.dump(hashmap, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [84]:
len(hashmap)

22193

In [85]:
ids = pd.DataFrame(ids)
ids = ids.reindex(concats.index)
ids = ids.dropna()

In [86]:
concats = concats.fillna(0.0)

In [87]:
ids_m = pd.DataFrame()
ids_m["_id"] = concats.iloc[:,[1,2,3]].apply(lambda row: hash_match(row.iloc[0],row.iloc[1],row.iloc[2],1), axis = 1)

In [88]:
concats['AAG'] = np.nan
concats['AAS'] = np.nan
concats['AASO'] = np.nan
concats['AAF'] = np.nan
concats['AAC'] = np.nan
concats['AAY'] = np.nan
concats['AAR'] = np.nan

concats['HAG'] = np.nan
concats['HAS'] = np.nan
concats['HASO'] = np.nan
concats['HAF'] = np.nan
concats['HAC'] = np.nan
concats['HAY'] = np.nan
concats['HAR'] = np.nan

for i in ids.index:
    #awaypart
    ids_away = ids[0][i][0]
    avg_goals = 0
    avg_shots = 0
    avg_shots_on = 0
    avg_fouls = 0
    avg_corners = 0
    avg_yellow = 0
    avg_red = 0
    for k in ids_away:
        if(concats.loc[i,"AwayTeam"] == concats.loc[k,"AwayTeam"]):
            avg_goals += concats.loc[k,"FTAG"]
            avg_shots += concats.loc[k,"AS"]
            avg_shots_on += concats.loc[k,"AST"]
            avg_fouls += concats.loc[k,"AF"]
            avg_corners = concats.loc[k,"AC"]
            avg_yellow += concats.loc[k,"AY"]
            avg_red += concats.loc[k,"AR"]
        else:
            avg_goals += concats.loc[k,"FTHG"]
            avg_shots += concats.loc[k,"HS"]
            avg_shots_on += concats.loc[k,"HST"]
            avg_fouls += concats.loc[k,"HF"]
            avg_corners = concats.loc[k,"HC"]
            avg_yellow += concats.loc[k,"HY"]
            avg_red += concats.loc[k,"HR"]
            
    concats.loc[i,'AAG'] = avg_goals
    concats.loc[i,'AAS'] = avg_shots
    concats.loc[i,'AASO'] = avg_shots_on
    concats.loc[i,'AAF'] = avg_fouls
    concats.loc[i,'AAC'] = avg_corners
    concats.loc[i,'AAY'] = avg_yellow
    concats.loc[i,'AAR'] = avg_red

            
    ids_home = ids[0][i][1]
    avg_goals = 0
    avg_shots = 0
    avg_shots_on = 0
    avg_fouls = 0
    avg_corners = 0
    avg_yellow = 0
    avg_red = 0
    for k in ids_home:
        if(concats.loc[i,"HomeTeam"] == concats.loc[k,"AwayTeam"]):
            avg_goals += concats.loc[k,"FTAG"]
            avg_shots += concats.loc[k,"AS"]
            avg_shots_on += concats.loc[k,"AST"]
            avg_fouls += concats.loc[k,"AF"]
            avg_corners = concats.loc[k,"AC"]
            avg_yellow += concats.loc[k,"AY"]
            avg_red += concats.loc[k,"AR"]
        else:
            avg_goals += concats.loc[k,"FTHG"]
            avg_shots += concats.loc[k,"HS"]
            avg_shots_on += concats.loc[k,"HST"]
            avg_fouls += concats.loc[k,"HF"]
            avg_corners = concats.loc[k,"HC"]
            avg_yellow += concats.loc[k,"HY"]
            avg_red += concats.loc[k,"HR"]
        
       
    concats.loc[i,'HAG'] = avg_goals
    concats.loc[i,'HAS'] = avg_shots
    concats.loc[i,'HASO'] = avg_shots_on
    concats.loc[i,'HAF'] = avg_fouls
    concats.loc[i,'HAC'] = avg_corners
    concats.loc[i,'HAY'] = avg_yellow
    concats.loc[i,'HAR'] = avg_red

In [89]:
concats = concats.drop([x for x in concats.index.values if x not in ids.index.values])

In [90]:
concats = concats.drop(columns = concats.columns[4:6])
concats = concats.drop(columns = concats.columns[5:20])

In [91]:
normalized_df=(concats[concats.columns[6:]]-concats[concats.columns[6:]].mean())/concats[concats.columns[6:]].std()

In [92]:
concats[concats.columns[6:]] = normalized_df

In [93]:
concats.columns[6:13]

Index(['AAG', 'AAS', 'AASO', 'AAF', 'AAC', 'AAY', 'AAR'], dtype='object')

In [94]:
concats.columns

Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTR', 'ID_FTR', 'AAG', 'AAS',
       'AASO', 'AAF', 'AAC', 'AAY', 'AAR', 'HAG', 'HAS', 'HASO', 'HAF', 'HAC',
       'HAY', 'HAR'],
      dtype='object')

In [95]:
concats.columns[13:]

Index(['HAG', 'HAS', 'HASO', 'HAF', 'HAC', 'HAY', 'HAR'], dtype='object')

In [96]:
def remove_accents(s):
    return ''.join((c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn'))

In [97]:
ids_team = list()
names = concats["HomeTeam"].drop_duplicates()
for i in names:
    df = np.array(concats[concats["HomeTeam"] == i].index, concats[concats["AwayTeam"] == i].index)
    ids_team.append(max(df))

values = list()
for i in range(len(names)):
    if(concats.loc[ids_team[i], "HomeTeam"] == names.iloc[i]):
        values.append(concats.loc[ids_team[i], concats.columns[13:]].values.astype(float))
    else:
        values.append(concats.loc[ids_team[i], concats.columns[6:13]].values.astype(float))
        
team_data = pd.DataFrame(values)
team_data = team_data.rename(columns = {0:"a",1:"b",2:"c",3:"d",4:"e",5:"f",6:"g"})
team_data.insert(0, '_id',  names.apply(lambda x : remove_accents(x).replace(" ", "_" ).lower()).values) 

In [98]:
concats.insert(0, '_id',  ids_m["_id"])

In [99]:
teams = pd.DataFrame(concats["HomeTeam"].drop_duplicates())
teams = teams.rename(columns = {"HomeTeam" : "football.co.uk"})

In [102]:
def update_db(data, coll):
    for index, row in data.iterrows():
        coll.replace_one({'_id':row['_id']}, row.to_dict(), True)

In [103]:
def fetch_old(name_post):
    client = pymongo.MongoClient("mongodb://plp_mongodb:27017")
    db = client["plp"][name_post]
    a= list(db["match"].find({}))
    a = pd.DataFrame(a)
    return a

In [104]:
def remove_match_db(id):
    query = { "_id": id}
    coll.delete_one(query)    

In [105]:
def pandas_to_mongo(df, name_post):
    client = pymongo.MongoClient("mongodb://plp_mongodb:27017")
    coll = client["plp"][name_post]
    old = fetch_old(name_post)
    new = df

    if len(old)==0:
        old = new
    update_filtered = new[new["_id"].isin(old["_id"])]

    update_db(update_filtered,coll)
    
    print("Done.")
    print("Updated : "+str(len(update_filtered)))
    print("Total matches in base : "+str(coll.estimated_document_count()))
        

In [106]:
team_data

,_id,a,b,c,d,e,f,g
0,nantes,1.467901,0.563369,1.240303,0.429381,0.530643,-0.596982,-0.423557
1,lille,-0.664917,0.220746,0.080627,0.271846,0.530643,0.017570,-0.423557
2,reims,0.614774,-0.072932,0.724892,-1.408532,-0.018166,0.017570,-0.423557
3,bastia,1.467901,0.661262,0.467186,0.219334,0.256239,0.017570,1.775221
4,evian_thonon_gaillard,-0.664917,-1.443427,-1.336755,-1.408532,-1.115782,-1.211534,-0.423557
...,...,...,...,...,...,...,...,...
254,venezia,0.614774,1.150724,1.369156,0.376869,1.353855,2.168501,3.973998
255,cosenza,0.614774,1.346509,1.111450,1.112034,0.530643,1.861225,1.775221
256,padova,1.041337,0.220746,0.596039,0.639428,1.079451,0.017570,1.775221
257,pordenone,-0.664917,0.563369,0.853745,1.007011,-0.018166,1.246673,1.775221


In [109]:
pandas_to_mongo(team_data,"team")
pandas_to_mongo(concats,"match")

Done.
Updated : 259
Total matches in base : 259
Done.
Updated : 21284
Total matches in base : 21284
